# Calibrating and Time Binning Astronomical Data

このNotebookは以下のNotebookを日本語訳したものです。自分でわかる様に意訳をしているため、ところどころ原文と表現が違うところがありますがご了承ください。  
※ 本Notebookは自分がコンペの理解を深めることを目的として作成しています。  
　 間違っているところが多くあるかと思いますので、お気軽にコメントください！修正します。

https://www.kaggle.com/code/gordonyip/update-calibrating-and-binning-astronomical-data

## Importing Package

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import os
import glob
import itertools
import gc
from astropy.stats import sigma_clip

from matplotlib import pyplot as plt

from tqdm import tqdm

## Define path of data, out

In [ ]:
path_folder = '/kaggle/input/ariel-data-challenge-2024/' # path to the folder containing the data
path_out = '/kaggle/tmp/data_light_raw/' # path to the folder to store the light data
output_dir = '/kaggle/tmp/data_light_raw/' # path for the output directory

# path_folder = 'ariel-data-challenge-2024/' # path to the local folder containing the data
# path_out = 'tmp/data_light_raw/' # path to the folder to store the light data
# output_dir = 'tmp/data_light_raw/' # path for the output directory

In [ ]:
if not os.path.exists(path_out):
    os.makedirs(path_out)
    print(f"Directory {path_out} created.")
else:
    print(f"Directory {path_out} already exists.")

## Data Preprocessing

### Define chunk_size to avoid exceeding the memory capacity

In [ ]:
CHUNKS_SIZE = 6 

### Step 1: Analog-to-Digital Conversion
アナログーデジタル変換(ADC)は、ピクセルの電圧を整数値に変換するために検出機によって行われる。この操作を戻すために、キャリブレーションファイルである「train_adc_info.csv」に含まれるゲインとオフセットを使用する。

オフセット誤差、ゲイン誤差を有効にした場合の値は以下の様に求められる。

$$
補正後の値 = (\text{補正前の値} - \text{offset}) \times \text{gain}
$$
今回は補正前の値が欲しいので、以下の関数で計算している。

In [ ]:
def ADC_convert(signal, gain, offset):
    signal = signal.astype(np.float64)
    signal /= gain
    signal += offset
    return signal

### Step 2: Mask hot/dead pixel
Dead pixel(pixel error) : 1つのフレームにある１個以上のプクセルが、収録した画像を正しく表示しない状態。
→正しく表示できないので、前処理でマスクすべき。



In [ ]:
def mask_hot_dead(signal, dead, dark):
    hot = sigma_clip(
        dark, sigma=5, maxiters=5
    ).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))
    signal = np.ma.masked_where(dead, signal)
    signal = np.ma.masked_where(hot, signal)
    return signal

### Step 3: Linearity Correction
#### ピクセル応答の非線形性について
ピクセル応答は本来線形性を持つ。以下の要因でピクセル応答が非線形となりうる。
1. 容量漏れ（Capacity Leakage）
    多くの電子が集まった時に、キャパシタにかかる電圧も大きくなる。電圧の容量が想定より大きくなった場合に応答が非線形になりうる。
1. 飽和効果（Saturation Effect）
    ウェル（光を受けて電子が集まる場所）が飽和状態となった時に、光子によって生じる電子が十分に蓄積されず、ピクセル応答が鈍くなる。→応答が非線形になる。
他にも、量子効率の変動、温度依存性などがある。

これらの非線形効果を補正するために以下の関数を定義する。


In [ ]:
def apply_linear_corr(linear_corr,clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

### Step ４: Dark Current Subtraction

ダークカレントは、入射光とは無関係に積分時間中に各ピクセルに蓄積される一定の信号を表す。積分時間に比例してダークカレントが蓄積して行く。

提供されたデータには、ダークフレームやデッドピクセルマップなどのキャリブレーションファイルが含まれている。これらを使用して観測データを前処理することができます。ダークフレームは、非常に短い露光時間で撮影された検出器の応答を示すマップであり、これを用いて検出器のダークカレントを補正する。

補正された画像は、次のようにして得る。

$$
\text{image} - \text{dark} \times \Delta t
$$

ここで、ダークカレントマップは最初にデッドピクセルの補正が行われる。


In [ ]:
def clean_dark(signal, dead, dark, dt):

    dark = np.ma.masked_where(dead, dark)
    dark = np.tile(dark, (signal.shape[0], 1, 1))

    signal -= dark* dt[:, np.newaxis, np.newaxis]
    return signal

### Step ５: Get Correlated Double Sampling (CDS)
サイエンスフレームは、露光の開始時と終了時の間で交互に取得される。読み出しの方式は、二重サンプリングを伴うランプであり、これをCorrelated Double Sampling (CDS)と呼ぶ。検出器は、露光の開始時と終了時の2回読み出す。最終的なCDSは、以下の差分として得られる。

$$
\text{CDS} = (\text{End of exposure}) - (\text{Start of exposure})
$$

疑問  
サイエンスフレームってなに？？別名があるんじゃないか？

In [ ]:
def get_cds(signal):
    cds = signal[:,1::2,:,:] - signal[:,::2,:,:]
    return cds

### Step ６ (Optional): Time Binning
このステップは主にメモリを節約するために実行される。  
時系列観測データが指定された頻度でまとめてビンにされる。

→メモリが足りないときはこちらを適用すると、学習に使うことができる様になるかも？？

In [ ]:
def bin_obs(cds_signal,binning):
    cds_transposed = cds_signal.transpose(0,1,3,2)
    cds_binned = np.zeros((cds_transposed.shape[0], cds_transposed.shape[1]//binning, cds_transposed.shape[2], cds_transposed.shape[3]))
    for i in range(cds_transposed.shape[1]//binning):
        cds_binned[:,i,:,:] = np.sum(cds_transposed[:,i*binning:(i+1)*binning,:,:], axis=1)
    return cds_binned

### Step 7: Flat Field Correction
フラットフィールドは、検出器の均一な光子に対する応答を示すマップであり、ピクセルごとの検出器の変動、例えば各ピクセルの異なる量子効率を補正するために使用される。


In [ ]:
def correct_flat_field(flat,dead, signal):
    flat = flat.transpose(1, 0)
    dead = dead.transpose(1, 0)
    flat = np.ma.masked_where(dead, flat)
    flat = np.tile(flat, (signal.shape[0], 1, 1))
    signal = signal / flat
    return signal

## Calibrating Single training data
### 前処理の方針
データの処理については、  
- ピクセル応答の非線形性を補正する
- フラットフィールド、ダークマップ、デッドピクセルマップを適用する
- データをそのまま変更せずに残すか  
を選択できる。  

観測データは、AIRSでは30フレーム、FGS1では360フレームごとに時間的にビンニングされ、より軽量で使いやすいデータキューブが得られる。画像は、波長軸に沿ってピクセル39から321の間でカットされ、波長次元に残った282ピクセルが、AIRSからの最後の282ターゲットポイントに一致するように調整される。283番目のターゲットポイントはFGS1のもので、後で追加される。

In [ ]:
## we will start by getting the index of the training data:
def get_index(files,CHUNKS_SIZE ):
    index = []
    for file in files :
        file_name = file.split('/')[-1]
        if file_name.split('_')[0] == 'AIRS-CH0' and file_name.split('_')[1] == 'signal.parquet':
            file_index = os.path.basename(os.path.dirname(file))
            index.append(int(file_index))
    index = np.array(index)
    index = np.sort(index) 
    # credit to DennisSakva
    index=np.array_split(index, len(index)//CHUNKS_SIZE)
    
    return index

In [ ]:
files = glob.glob(os.path.join(path_folder + 'train/', '*/*'))
# print(files)

index = get_index(files[:24],CHUNKS_SIZE)  ## 48 is hardcoded here but please feel free to remove it if you want to do it for the entire dataset


In [ ]:
DO_MASK = True
DO_THE_NL_CORR = False
DO_DARK = True
DO_FLAT = True
TIME_BINNING = True

cut_inf, cut_sup = 39, 321
l = cut_sup - cut_inf

train_adc_info = pl.read_csv(os.path.join(path_folder, 'train_adc_info.csv'))
axis_info = pl.read_parquet(os.path.join(path_folder,'axis_info.parquet'))

AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l))
FGS1_clean = np.zeros((CHUNKS_SIZE, 135000, 32, 32))

i = 0

#### AIRS-CH0_signal

In [ ]:
planet_id = index[0][0].item()


df = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_signal.parquet'))
signal = df.to_numpy().reshape((df.shape[0], 32, 356))
plt.title('Natural image')
plt.imshow(signal[0,:,:])
plt.show()

gain = train_adc_info.filter(pl.col('planet_id') == planet_id).select(['AIRS-CH0_adc_gain']).to_numpy()
offset = train_adc_info.filter(pl.col('planet_id') == planet_id).select(['AIRS-CH0_adc_offset']).to_numpy()

# AD Convert を復元している。
signal = ADC_convert(signal, gain, offset)
plt.title('After ADC_convert')
plt.imshow(signal[0,:,:])
plt.show()

# 積分時間を取ってきている。
dt_airs = axis_info['AIRS-CH0-integration_time'].drop_nulls().to_numpy()
chopped_signal = signal[:, :, cut_inf:cut_sup]
plt.title('Chopped Signal')
plt.imshow(chopped_signal[0,:,:])
plt.show()
del signal, df

In [ ]:
# CLEANING THE DATA: AIRS
flat = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/flat.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
dark = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/dark.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
dead_airs = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/dead.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
linear_corr = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/linear_corr.parquet')).to_numpy().reshape((6, 32, 356))[:, :, cut_inf:cut_sup]


In [ ]:
if DO_MASK:
    chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)
    plt.title('Masked Chopped Signal')
    plt.imshow(chopped_signal[0,:,:])
    plt.show()
    AIRS_CH0_clean[i] = chopped_signal
else:
    AIRS_CH0_clean[i] = chopped_signal
    
if DO_THE_NL_CORR: 
    linear_corr_signal = apply_linear_corr(linear_corr,AIRS_CH0_clean[i])
    plt.title('Linear corr chopped Signal')
    plt.imshow(linear_corr_signal[0,:,:])
    plt.show()
    AIRS_CH0_clean[i,:, :, :] = linear_corr_signal
del linear_corr

if DO_DARK: 
    cleaned_signal = clean_dark(AIRS_CH0_clean[i], dead_airs, dark, dt_airs)
    AIRS_CH0_clean[i] = cleaned_signal
    plt.title('Darked chopped Signal')
    plt.imshow(cleaned_signal[0,:,:])
    plt.show()
else: 
    pass
del dark

#### FGS1_signal

In [ ]:
df = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_signal.parquet'))
fgs_signal = df.to_numpy().reshape((df.shape[0], 32, 32))
plt.title('Natural image')
plt.imshow(fgs_signal[0,:,:])
plt.show()

FGS1_gain = train_adc_info.filter(pl.col('planet_id') == planet_id).select(['FGS1_adc_gain']).to_numpy()
FGS1_offset = train_adc_info.filter(pl.col('planet_id') == planet_id).select(['FGS1_adc_offset']).to_numpy()
fgs_signal = ADC_convert(fgs_signal, FGS1_gain, FGS1_offset)
plt.title('After ADC_convert')
plt.imshow(fgs_signal[0,:,:])
plt.show()

dt_fgs1 = np.ones(len(fgs_signal))*0.1
chopped_FGS1 = fgs_signal
plt.title('Chopped Signal')
plt.imshow(chopped_FGS1[0,:,:])
plt.show()
del fgs_signal, df

In [ ]:
# CLEANING THE DATA: FGS1
flat = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/flat.parquet')).to_numpy().reshape((32, 32))
dark = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/dark.parquet')).to_numpy().reshape((32, 32))
dead_fgs1 = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/dead.parquet')).to_numpy().reshape((32, 32))
linear_corr = pl.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/linear_corr.parquet')).to_numpy().reshape((6, 32, 32))

In [ ]:
if DO_MASK:
    chopped_FGS1 = mask_hot_dead(chopped_FGS1, dead_fgs1, dark)
    plt.title('Masked Chopped Signal')
    plt.imshow(chopped_FGS1[0,:,:])
    plt.show()
    FGS1_clean[i] = chopped_FGS1
else:
    FGS1_clean[i] = chopped_FGS1

if DO_THE_NL_CORR: 
    linear_corr_signal = apply_linear_corr(linear_corr,FGS1_clean[i])
    plt.title('Linear corr chopped Signal')
    plt.imshow(linear_corr_signal[0,:,:])
    plt.show()
    FGS1_clean[i,:, :, :] = linear_corr_signal
del linear_corr

if DO_DARK: 
    cleaned_signal = clean_dark(FGS1_clean[i], dead_fgs1, dark,dt_fgs1)
    plt.title('Darked chopped Signal')
    plt.imshow(cleaned_signal[0,:,:])
    plt.show()
    FGS1_clean[i] = cleaned_signal
else: 
    pass
del dark

gc.collect()

In [ ]:
for n, index_chunk in enumerate(tqdm(index)):
    AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l))
    FGS1_clean = np.zeros((CHUNKS_SIZE, 135000, 32, 32))
    
    for i in range (CHUNKS_SIZE) : 
        df = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_signal.parquet'))
        signal = df.to_numpy().reshape((df.shape[0], 32, 356))
        gain = train_adc_info.filter(pl.col('planet_id') == index_chunk[i]).select(['AIRS-CH0_adc_gain']).to_numpy()
        offset = train_adc_info.filter(pl.col('planet_id') == index_chunk[i]).select(['AIRS-CH0_adc_offset']).to_numpy()
        signal = ADC_convert(signal, gain, offset)
        dt_airs = axis_info['AIRS-CH0-integration_time'].drop_nulls().to_numpy()
        chopped_signal = signal[:, :, cut_inf:cut_sup]
        del signal, df
        
        # CLEANING THE DATA: AIRS
        flat = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_calibration/flat.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
        dark = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_calibration/dark.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
        dead_airs = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_calibration/dead.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
        linear_corr = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_calibration/linear_corr.parquet')).to_numpy().reshape((6, 32, 356))[:, :, cut_inf:cut_sup]
        
        if DO_MASK:
            chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)
            AIRS_CH0_clean[i] = chopped_signal
        else:
            AIRS_CH0_clean[i] = chopped_signal
            
        if DO_THE_NL_CORR: 
            linear_corr_signal = apply_linear_corr(linear_corr,AIRS_CH0_clean[i])
            AIRS_CH0_clean[i,:, :, :] = linear_corr_signal
        del linear_corr
        
        if DO_DARK: 
            cleaned_signal = clean_dark(AIRS_CH0_clean[i], dead_airs, dark, dt_airs)
            AIRS_CH0_clean[i] = cleaned_signal
        else: 
            pass
        del dark
        
        df = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_signal.parquet'))
        fgs_signal = df.to_numpy().reshape((df.shape[0], 32, 32))
        
        FGS1_gain = train_adc_info.filter(pl.col('planet_id') == index_chunk[i]).select(['FGS1_adc_gain']).to_numpy()
        FGS1_offset = train_adc_info.filter(pl.col('planet_id') == index_chunk[i]).select(['FGS1_adc_offset']).to_numpy()
        
        fgs_signal = ADC_convert(fgs_signal, FGS1_gain, FGS1_offset)
        dt_fgs1 = np.ones(len(fgs_signal))*0.1
        chopped_FGS1 = fgs_signal
        del fgs_signal, df
        
        # CLEANING THE DATA: FGS1
        flat = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_calibration/flat.parquet')).to_numpy().reshape((32, 32))
        dark = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_calibration/dark.parquet')).to_numpy().reshape((32, 32))
        dead_fgs1 = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_calibration/dead.parquet')).to_numpy().reshape((32, 32))
        linear_corr = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_calibration/linear_corr.parquet')).to_numpy().reshape((6, 32, 32))
        
        if DO_MASK:
            chopped_FGS1 = mask_hot_dead(chopped_FGS1, dead_fgs1, dark)
            FGS1_clean[i] = chopped_FGS1
        else:
            FGS1_clean[i] = chopped_FGS1

        if DO_THE_NL_CORR: 
            linear_corr_signal = apply_linear_corr(linear_corr,FGS1_clean[i])
            FGS1_clean[i,:, :, :] = linear_corr_signal
        del linear_corr
        
        if DO_DARK: 
            cleaned_signal = clean_dark(FGS1_clean[i], dead_fgs1, dark,dt_fgs1)
            FGS1_clean[i] = cleaned_signal
        else: 
            pass
        del dark
        
    # SAVE DATA AND FREE SPACE
    AIRS_cds = get_cds(AIRS_CH0_clean)
    FGS1_cds = get_cds(FGS1_clean)
    
    del AIRS_CH0_clean, FGS1_clean
    
    ## (Optional) Time Binning to reduce space
    if TIME_BINNING:
        AIRS_cds_binned = bin_obs(AIRS_cds,binning=30)
        FGS1_cds_binned = bin_obs(FGS1_cds,binning=30*12)
    else:
        AIRS_cds = AIRS_cds.transpose(0,1,3,2) ## this is important to make it consistent for flat fielding, but you can always change it
        AIRS_cds_binned = AIRS_cds
        FGS1_cds = FGS1_cds.transpose(0,1,3,2)
        FGS1_cds_binned = FGS1_cds
    
    del AIRS_cds, FGS1_cds
    
    for i in range (CHUNKS_SIZE):
        flat_airs = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/AIRS-CH0_calibration/flat.parquet')).to_numpy().reshape((32, 356))[:, cut_inf:cut_sup]
        flat_fgs = pl.read_parquet(os.path.join(path_folder,f'train/{index_chunk[i]}/FGS1_calibration/flat.parquet')).to_numpy().reshape((32, 32))
        if DO_FLAT:
            corrected_AIRS_cds_binned = correct_flat_field(flat_airs,dead_airs, AIRS_cds_binned[i])
            AIRS_cds_binned[i] = corrected_AIRS_cds_binned
            corrected_FGS1_cds_binned = correct_flat_field(flat_fgs,dead_fgs1, FGS1_cds_binned[i])
            FGS1_cds_binned[i] = corrected_FGS1_cds_binned
        else:
            pass
    # print(gc.collect())
    
    ## save data
    np.save(os.path.join(path_out, 'AIRS_clean_train_{}.npy'.format(n)), AIRS_cds_binned)
    np.save(os.path.join(path_out, 'FGS1_train_{}.npy'.format(n)), FGS1_cds_binned)
    del AIRS_cds_binned
    del FGS1_cds_binned